In [ ]:
# Importing Libraries
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Activation, Dropout
from keras.callbacks import ModelCheckpoint
from keras.datasets import mnist
from keras.utils import to_categorical
import tensorflow as tf
from keras.models import Model

In [ ]:
# AlexNet Model
def build_alexnet(input_shape, num_classes):
    model = Sequential([
        # Layer 1
        Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'), # Adjusted stride and added padding

        # Layer 2
        Conv2D(256, (5, 5), padding='same', activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'), # Adjusted stride and added padding

        # Layer 3
        Conv2D(384, (3, 3), padding='same', activation='relu'),

        # Layer 4
        Conv2D(384, (3, 3), padding='same', activation='relu'),

        # Layer 5
        Conv2D(256, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'), # Adjusted stride and added padding

        Flatten(),

        # Layer 6
        Dense(4096, activation='relu'),
        Dropout(0.5),

        # Layer 7
        Dense(4096, activation='relu'),
        Dropout(0.5),

        # Output layer
        Dense(num_classes, activation='softmax')
    ])

    return model

In [ ]:
# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Preprocess the data
input_shape = (28, 28, 1)
num_classes = 10
train_images = train_images.reshape(-1, 28, 28, 1).astype('float32') / 255.0
test_images = test_images.reshape(-1, 28, 28, 1).astype('float32') / 255.0
train_labels = to_categorical(train_labels, num_classes)
test_labels = to_categorical(test_labels, num_classes)

# Split the training data into training and validation sets
from sklearn.model_selection import train_test_split
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.1, random_state=42)

# Build the AlexNet-like model for MNIST
model = build_alexnet(input_shape, num_classes)
model.summary()

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define the ModelCheckpoint callback to save the best model
checkpoint_filepath = 'alexnet.h5'
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,
                                            save_best_only=True,
                                            monitor='val_accuracy',
                                            mode='max',
                                            verbose=1)

# Fit the model to the training data with validation data and ModelCheckpoint callback
history = model.fit(train_images, train_labels, epochs=10, batch_size=128, validation_data=(val_images, val_labels),
                    callbacks=[model_checkpoint_callback])

# Define a new model without the output layer
output_layer_index = -2  # Index of the layer before the output layer
feature_extractor = Model(inputs=model.inputs, outputs=model.layers[output_layer_index].output)

# Save the model architecture and weights
feature_extractor.save('alexnet_feature_extractor.h5')

# Evaluate the best model on the test data
best_model = tf.keras.models.load_model(checkpoint_filepath)
test_loss, test_accuracy = best_model.evaluate(test_images, test_labels, verbose=2)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 5, 5, 96)          11712     
                                                                 
 batch_normalization_3 (Bat  (None, 5, 5, 96)          384       
 chNormalization)                                                
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 3, 3, 96)          0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 3, 3, 256)         614656    
                                                                 
 batch_normalization_4 (Bat  (None, 3, 3, 256)         1024      
 chNormalization)                                                
                                                      

313/313 - 2s - loss: 0.0608 - accuracy: 0.9850 - 2s/epoch - 6ms/step
Test Loss: 0.0608
Test Accuracy: 0.9850
